# **Modulo 39 - XGBOOST**
**Aula 3: Aplicando com Python**

In [1]:
! pip install xgboost

   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   ----- ---------------------------------- 8.1/56.8 MB 47.7 MB/s eta 0:00:02
   ------------------- -------------------- 28.3/56.8 MB 74.8 MB/s eta 0:00:01
   --------------------------------- ------ 47.4/56.8 MB 81.8 MB/s eta 0:00:01
   ---------------------------------------- 56.8/56.8 MB 76.0 MB/s  0:00:00


In [2]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

**XGBoost:** A biblioteca principal que fornece a implementação do algoritmo XGBoost.

**NumPy:** Biblioteca fundamental para computação científica em Python, usada para manipulação de arrays e operações matemáticas.

**Pandas:** Biblioteca para manipulação e análise de dados, útil para carregar e preparar dados.

**Scikit-learn:** Biblioteca que fornece ferramentas para pré-processamento de dados, divisão de conjuntos de dados, avaliação de modelos e técnicas de ajuste de hiperparâmetros.

**Matplotlib e/ou Seaborn:** Bibliotecas para visualização de dados, úteis para criar gráficos e plots, incluindo a visualização da importância das características.


# Continuaremos com nosso problema de classificação mas o Xgboost, assim como árvore e floresta, também funciona na regressão.

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse'
)

model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss'
)

In [3]:
base = pd.read_csv('fetal_health.csv')

In [4]:
base

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.000,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.000,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.000,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.000,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,140.0,0.000,0.000,0.007,0.000,0.0,0.0,79.0,0.2,25.0,...,137.0,177.0,4.0,0.0,153.0,150.0,152.0,2.0,0.0,2.0
2122,140.0,0.001,0.000,0.007,0.000,0.0,0.0,78.0,0.4,22.0,...,103.0,169.0,6.0,0.0,152.0,148.0,151.0,3.0,1.0,2.0
2123,140.0,0.001,0.000,0.007,0.000,0.0,0.0,79.0,0.4,20.0,...,103.0,170.0,5.0,0.0,153.0,148.0,152.0,4.0,1.0,2.0
2124,140.0,0.001,0.000,0.006,0.000,0.0,0.0,78.0,0.4,27.0,...,103.0,169.0,6.0,0.0,152.0,147.0,151.0,4.0,1.0,2.0


In [5]:
##Separando a variável dependente Y e as variáveis independentes X
X = base.drop(columns=['fetal_health'])
Y = base['fetal_health']

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Documentação de Parametros: [LINK](https://xgboost.readthedocs.io/en/stable/parameter.html)

In [7]:
model_xgboost = xgb.XGBClassifier()
model_xgboost.objective

'binary:logistic'

Por padrão o xgboost espera uma classificação binária. Porém ao identificar mais classes, ele se ajusta nas previsões.

In [8]:
model_xgboost = xgb.XGBClassifier().fit(X_train, Y_train)
model_xgboost.objective

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2], got [1. 2. 3.]

Se nossas classes são representadas como valores flutuantes (por exemplo, [1.0, 2.0, 3.0]), precisamos converter esses valores para inteiros consecutivos começando por 0.

In [9]:
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_encoded, test_size=0.2, random_state=42)

In [11]:
model_xgboost = xgb.XGBClassifier().fit(X_train, Y_train)
model_xgboost.objective

'multi:softprob'

O objetivo multi:softprob em XGBoost é usado para problemas de classificação multiclasse onde você deseja obter as probabilidades de cada classe em vez de apenas a classe com a maior probabilidade.

#**Modulo 39 - XGBOOST**
**Aula 4: Aplicando com Python**

# Mas como ficaria nosso modelo com hyperparametros?

In [12]:
model_xgboost_hyper = xgb.XGBClassifier(
    max_depth=6,                  # Profundidade máxima da árvore
    n_estimators=100,             # Número de árvores a serem construídas

)

In [13]:
model_xgboost_hyper = xgb.XGBClassifier().fit(X_train, Y_train)

In [14]:
Y_pred = model_xgboost.predict(X_test)           # Previsões das classes
Y_pred_prob = model_xgboost.predict_proba(X_test)  # Probabilidades de cada classe

A utilização de predict() e predict_proba() em modelos como o XGBoost (e outros modelos de aprendizado de máquina) pode oferecer diferentes perspectivas sobre as previsões feitas pelo modelo.

predict_proba()
O que faz: Retorna as probabilidades associadas a cada classe para as instâncias do conjunto de teste. Para cada amostra, retorna um array de probabilidades, onde cada valor indica a probabilidade de a amostra pertencer a cada classe.

In [14]:
Y_pred_prob

array([[9.9914527e-01, 8.2551420e-04, 2.9218938e-05],
       [9.9962473e-01, 6.3117972e-05, 3.1216606e-04],
       [9.9917692e-01, 6.9811649e-04, 1.2491069e-04],
       ...,
       [9.9996769e-01, 2.8443412e-05, 3.8518756e-06],
       [9.9920815e-01, 2.3023431e-04, 5.6159776e-04],
       [1.5883340e-01, 8.4045565e-01, 7.1096781e-04]], dtype=float32)

**[9.9914527e-01]: Aproximadamente 0.9991, que é a probabilidade de a amostra pertencer à primeira classe (classe 0)**

**[8.2551420e-04]: Aproximadamente 0.0008, que é a probabilidade de a amostra pertencer à segunda classe (classe 1).**

**[2.9218938e-05]: Aproximadamente 0.00003, que é a probabilidade de a amostra pertencer à terceira classe (classe 2).**

In [15]:
# Então uma vez que temos as probabilidades, fazemos a transformação das previsões e os rótulos de volta para os rótulos originais
Y_pred_original = label_encoder.inverse_transform(Y_pred)
Y_test_original = label_encoder.inverse_transform(Y_test)

In [16]:
accuracy = accuracy_score(Y_test_original, Y_pred_original)
report = classification_report(Y_test_original, Y_pred_original)
conf_matrix = confusion_matrix(Y_test_original, Y_pred_original)

print(f'Acurácia: {accuracy}')
print('Relatório de Classificação:')
print(report)
print('Matriz de Confusão:')
print(conf_matrix)

Acurácia: 0.960093896713615
Relatório de Classificação:
              precision    recall  f1-score   support

         1.0       0.98      0.98      0.98       333
         2.0       0.89      0.86      0.87        64
         3.0       0.94      1.00      0.97        29

    accuracy                           0.96       426
   macro avg       0.93      0.95      0.94       426
weighted avg       0.96      0.96      0.96       426

Matriz de Confusão:
[[325   7   1]
 [  8  55   1]
 [  0   0  29]]


In [17]:
importances = model_xgboost.get_booster().get_score(importance_type='gain')

# Convertendo o dicionário de importâncias para um DataFrame
importance_df = pd.DataFrame(list(importances.items()), columns=['Feature', 'Importance'])
importance_df['Importance'] = importance_df['Importance'].astype(float)
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)

                                              Feature  Importance
7                mean_value_of_short_term_variability    3.982133
16                                     histogram_mean    3.418476
6                     abnormal_short_term_variability    1.850552
1                                       accelerations    1.712905
8   percentage_of_time_with_abnormal_long_term_var...    1.459023
5                            prolongued_decelerations    1.185336
0                                      baseline value    0.981800
15                                     histogram_mode    0.691523
3                                uterine_contractions    0.639723
13                          histogram_number_of_peaks    0.559857
14                         histogram_number_of_zeroes    0.533052
12                                      histogram_max    0.517150
11                                      histogram_min    0.367634
17                                   histogram_median    0.354567
18        

Com a métrica de "Gain", você  está visualizando a importância das características baseada na melhoria média que cada característica proporciona para o critério objetivo do modelo.

Ganho: Reflete a contribuição média da característica para a melhoria da função objetivo. Um valor maior indica que a característica proporciona uma maior redução no erro médio durante o treinamento das árvores.

mean_value_of_short_term_variability tem um ganho de 3.982133, indicando que é a característica mais relevante em termos de melhoria do critério objetivo.

histogram_mean com um ganho de 3.418476 é a segunda característica mais importante.

#**Modulo 39 - XGBOOST**
**Aula 5: Rodando o modelo com as melhores Features Importances**

Uma vez que encontramos então as variaveis com maior importancia, é interessante rodarmos nosso modelo com as principais e comparar o resultado com o modelo anterior. Essa é outra técnica que nos ajuda a diminuir a complexidade do modelo, reduzir overfitting e aumentar a perforamnce.

In [24]:
# Definindo o número de características a serem mantidas (por exemplo, as top 10)
num_features = 10

In [25]:
# Aqui selecionamos essas features como uma lista
top_features = importance_df.head(num_features)['Feature'].tolist()

In [26]:
# Criamos um novo DataFrame apenas com as top características
X_reduced = X[top_features]

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X_reduced, Y_encoded, test_size=0.2, random_state=42)

In [28]:
model_xgboost_reduced = xgb.XGBClassifier(
    max_depth=6,                  # Profundidade máxima da árvore
    n_estimators=100,             # Número de árvores a serem construídas

)

In [29]:
model_xgboost_reduced = xgb.XGBClassifier().fit(X_train, Y_train)

In [30]:
Y_pred = model_xgboost_reduced.predict(X_test)
Y_pred_prob = model_xgboost_reduced.predict_proba(X_test)

In [31]:
Y_pred_reduce = label_encoder.inverse_transform(Y_pred)
Y_test_reducec= label_encoder.inverse_transform(Y_test)

In [32]:
accuracy = accuracy_score(Y_test_reducec, Y_pred_reduce)
report = classification_report(Y_test_reducec, Y_pred_reduce)
conf_matrix = confusion_matrix(Y_test_reducec, Y_pred_reduce)

print(f'Acurácia: {accuracy}')
print('Relatório de Classificação:')
print(report)
print('Matriz de Confusão:')
print(conf_matrix)

Acurácia: 0.9624413145539906
Relatório de Classificação:
              precision    recall  f1-score   support

         1.0       0.98      0.98      0.98       333
         2.0       0.92      0.86      0.89        64
         3.0       0.91      1.00      0.95        29

    accuracy                           0.96       426
   macro avg       0.93      0.95      0.94       426
weighted avg       0.96      0.96      0.96       426

Matriz de Confusão:
[[326   5   2]
 [  8  55   1]
 [  0   0  29]]
